# Data Transformation: Lakeflow Spark Declarative Pipelines

## The Situation

Your leadership team just dropped a bombshell: they want dashboards, interactive "talk to my data" capabilities, predictive maintenance models, and AI agent systems - **all by the end of the week**. Your IoT sensors on planes are generating massive amounts of data, and you need to get it production-ready, fast.

Good news: Databricks has **Lakeflow Spark Declarative Pipelines** that can help you build reliable, production-ready data pipelines in minutes, not days.

---

## What You'll Learn

✅ What declarative pipelines are and why they matter  
✅ Create a streaming table to ingest and clean sensor data  
✅ Create a materialized view for aggregated metrics  
✅ Deploy your pipeline to production  

---

## What are Lakeflow Spark Declarative Pipelines?

Lakeflow Spark Declarative Pipelines (formerly Delta Live Tables) is a framework for building reliable, production-ready data pipelines. Instead of writing complex code to manage checkpoints, handle incremental processing, and track data quality, you simply declare **what** you want using either SQL or Python. The framework automatically handles:

- ✅ **Incremental processing** - Only process new/changed data
- ✅ **Dependency management** - Determine execution order automatically
- ✅ **Data quality** - Built-in validation with expectations
- ✅ **Monitoring** - Automatic lineage and observability
- ✅ **Recovery** - Checkpoint management and error handling

### Two Key Building Blocks

**1. Streaming Tables**
- For incremental, append-only data processing
- Perfect for ingesting raw sensor data and cleaning it
- Uses Auto Loader to automatically discover new files
- Each row processed exactly once (exactly-once semantics)

**2. Materialized Views**
- For aggregations that need to update (not just append)
- Perfect for business metrics and KPIs
- Always return correct, up-to-date results
- Automatically recompute when source data changes

### About This Tutorial

This tutorial explains how to create and deploy an ETL (extract, transform, and load) pipeline for data orchestration using Lakeflow Spark Declarative Pipelines and Auto Loader. You will use pipelines to:

- Ingest raw sensor data from a volume into a streaming table
- Transform and validate the data with data quality expectations
- Create aggregated views for dashboards and analytics
- Perform ad-hoc queries on the processed data
- Automate the ETL pipeline with a scheduled job

---

**Reference:** [Lakeflow Pipelines Documentation](https://learn.microsoft.com/en-us/azure/databricks/ldp/)

## Step 1: Create Your Pipeline

Follow these steps to create a new Lakeflow Spark Declarative Pipeline:

1. In your workspace, click **New** (Plus icon) in the sidebar, then select **ETL Pipeline**.
2. Give your pipeline a unique name: `IoT Sensor Pipeline`
3. Just below the name, select the default catalog and schema for the data that you generate:
   - **Catalog:** `dwx_airops_insights_platform_dev_working`
   - **Schema:** Your personal schema (should look like `jane_doe` - your username with special characters replaced with underscores)
   - Note: You must have `ALL PRIVILEGES` or `USE CATALOG` and `CREATE SCHEMA` permissions
4. For this tutorial, select **Start with an empty file**.
5. In **Folder path**, specify a location for your source files, or accept the default (your user folder).
6. Choose **Python** or **SQL** as the language for your first source file (a pipeline can mix and match languages, but each file must be in a single language).
7. Click **Select**.

The pipeline editor appears for the new pipeline. An empty source file for your language is created, ready for your first transformation.

### What You'll See

The **Lakeflow Pipelines Editor** has three main components:
- **Left panel**: Code editor where you write your pipeline logic
- **Right panel**: Pipeline graph (DAG) showing your data flow
- **Bottom panel**: Table preview and data results

---

**Note:** This tutorial uses serverless compute and Unity Catalog. For all configuration options that are not specified, use the default settings.

## Step 2: Develop Your Streaming Table

In this step, you will use the **Lakeflow Pipelines Editor** to develop and validate source code for the pipeline interactively.

The code uses **Auto Loader** for incremental data ingestion. Auto Loader automatically detects and processes new files as they arrive in cloud object storage.

A blank source code file is automatically created and configured for the pipeline. The file is created in the **transformations** folder of your pipeline. By default, all `*.py` and `*.sql` files in the transformations folder are part of the source for your pipeline.

Let's build a streaming table that:
1. Ingests raw sensor data from the volume using Auto Loader
2. Cleans the data (fixes negative air pressure values)
3. Validates data quality with expectations

This will be your **silver layer** - cleaned, validated sensor data ready for analysis.

### Instructions

1. Copy and paste the code below into your source file. Be sure to use the language that you selected for the file in Step 1.
2. Click Play icon **Run file** or **Run pipeline** to start an update for the connected pipeline.

When the update completes, the editor is updated with information about your pipeline:
- The **pipeline graph (DAG)**, in the sidebar to the right of your code, shows your `sensor_silver` table
- A **summary of the update** is shown at the top of the pipeline assets browser
- **Details of the tables** that were generated are shown in the bottom pane, and you can browse data from the tables by selecting one

---

### SQL Version

```sql
CREATE OR REFRESH STREAMING TABLE sensor_silver (
  CONSTRAINT valid_device_id EXPECT (device_id IS NOT NULL),
  CONSTRAINT valid_timestamp EXPECT (timestamp IS NOT NULL),
  CONSTRAINT valid_temperature_range EXPECT (temperature BETWEEN -50 AND 150),
  CONSTRAINT positive_air_pressure EXPECT (air_pressure > 0)
)
COMMENT 'Cleaned and validated aircraft sensor readings'
AS SELECT
  device_id,
  trip_id,
  factory_id,
  model_id,
  timestamp,
  airflow_rate,
  rotation_speed,
  CASE 
    WHEN air_pressure < 0 THEN ABS(air_pressure)
    ELSE air_pressure
  END as air_pressure,
  temperature,
  delay,
  density,
  current_timestamp() as processed_at
FROM STREAM read_files(
  '/Volumes/dwx_airops_insights_platform_dev_working/db_crash_course/sensor_data/',
  format => 'csv',
  header => 'true'
);
```



### Python Version

```python
from pyspark import pipelines as dp
from pyspark.sql.functions import col, when, abs as abs_func, current_timestamp

@dp.table(
    name="sensor_silver",
    comment="Cleaned and validated aircraft sensor readings",
    # Data quality rules - track violations
    expect={
        "valid_device_id": "device_id IS NOT NULL",
        "valid_timestamp": "timestamp IS NOT NULL",
        "valid_temperature_range": "temperature BETWEEN -50 AND 150"
    },
    # Drop rows that fail critical validation
    expect_or_drop={
        "positive_air_pressure": "air_pressure > 0"
    }
)
def sensor_silver():
    """
    Ingest and clean sensor data with Auto Loader.
    Auto Loader automatically handles:
    - Schema inference
    - New file discovery  
    - Exactly-once processing
    """
    return (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("header", "true")
        .load("/Volumes/dwx_airops_insights_platform_dev_working/db_crash_course/sensor_data/")
        # Fix data quality issue: negative air pressure
        .withColumn("air_pressure",
                   when(col("air_pressure") < 0, abs_func(col("air_pressure")))
                   .otherwise(col("air_pressure")))
        # Add processing timestamp
        .withColumn("processed_at", current_timestamp())
    )
```

---

### What's Happening Here

**For SQL version:**
- `CREATE OR REFRESH STREAMING TABLE` defines a table that processes streaming data
- `CONSTRAINT ... EXPECT` validates data quality (rows that fail are logged but not dropped)
- `STREAM read_files()` uses Auto Loader to automatically discover new CSV files
- `CASE WHEN` fixes negative air pressure values inline
- Framework handles all the checkpointing and incremental processing

**For Python version:**
- `@dp.table` decorator defines a streaming table
- `expect` tracks data quality violations (logs them but doesn't drop rows)
- `expect_or_drop` drops rows that fail critical validation
- Auto Loader (`cloudFiles`) automatically discovers new CSV files
- We fix negative air pressure values inline using `.withColumn()`
- Framework handles all the checkpointing and incremental processing

✅ **Result:** Clean, validated sensor data streaming into your silver table automatically as new files arrive!

## Step 3: Add Your Materialized View

Now let's create aggregated metrics that your dashboards and Genie spaces will use. We'll build a **gold layer** materialized view with factory-level KPIs.

Why a materialized view? Because we need aggregations that **update** when new data arrives (not just append).

### Instructions

You can add this query to the same source file, or create a new file to organize your pipeline code. The pipeline will process all queries together.

**To add to the same file:**
1. Scroll to the bottom of your existing source file
2. Copy and paste the code below
3. Click Play icon **Run file** or **Run pipeline** to update

**To create a new file (optional):**
1. From the pipeline assets browser sidebar, click Plus icon **Add** then **Source**
2. Choose the same language (SQL or Python) as your first file
3. Paste the code below
4. Click Play icon **Run file**

When the update completes:
- The **pipeline graph** now shows two tables: `sensor_silver` and `factory_kpis_gold`
- An edge connects them, showing `factory_kpis_gold` reads from `sensor_silver`
- You can select `factory_kpis_gold` in the bottom panel to preview the aggregated data

---

### SQL Version

```sql
CREATE OR REFRESH MATERIALIZED VIEW factory_kpis_gold
COMMENT 'Factory-level KPIs for aircraft maintenance dashboards'
AS SELECT
  s.factory_id,
  f.factory_name,
  f.region,
  f.city,
  COUNT(DISTINCT s.device_id) as total_devices,
  COUNT(*) as total_readings,
  ROUND(AVG(s.temperature), 2) as avg_temperature,
  ROUND(MAX(s.temperature), 2) as max_temperature,
  ROUND(AVG(s.air_pressure), 2) as avg_air_pressure,
  ROUND(AVG(s.rotation_speed), 2) as avg_rotation_speed
FROM sensor_silver s
LEFT JOIN dwx_airops_insights_platform_dev_working.db_crash_course.dim_factories f ON s.factory_id = f.factory_id
GROUP BY s.factory_id, f.factory_name, f.region, f.city;
```

### Python Version

```python
from pyspark.sql.functions import avg, max, count, countDistinct, round as spark_round

@dp.materialized_view(
    name="factory_kpis_gold",
    comment="Factory-level KPIs for aircraft maintenance dashboards"
)
def factory_kpis_gold():
    """
    Aggregate factory-level metrics.
    This materialized view automatically recomputes when new data arrives.
    Perfect for dashboards and reporting!
    """
    # Read from silver layer (cleaned data)
    sensors = spark.read.table("sensor_silver")
    
    # Join with factory dimension for context
    factories = spark.read.table("dwx_airops_insights_platform_dev_working.db_crash_course.dim_factories")
    
    enriched = sensors.join(factories, "factory_id", "left")
    
    # Calculate factory-level KPIs
    return (
        enriched
        .groupBy("factory_id", "factory_name", "region", "city")
        .agg(
            countDistinct("device_id").alias("total_devices"),
            count("*").alias("total_readings"),
            spark_round(avg("temperature"), 2).alias("avg_temperature"),
            spark_round(max("temperature"), 2).alias("max_temperature"),
            spark_round(avg("air_pressure"), 2).alias("avg_air_pressure"),
            spark_round(avg("rotation_speed"), 2).alias("avg_rotation_speed")
        )
    )
```

---

### What's Happening Here

**For SQL version:**
- `CREATE OR REFRESH MATERIALIZED VIEW` defines a view that automatically updates
- `LEFT JOIN` enriches sensor data with factory dimension data (name, region, city)
- `GROUP BY` aggregates metrics at the factory level
- `COUNT(DISTINCT ...)` counts unique devices, `AVG()` calculates averages
- The view automatically recomputes when new data arrives in `sensor_silver`

**For Python version:**
- `@dp.materialized_view` defines a view that auto-updates
- `spark.read.table("sensor_silver")` reads from the silver table (already cleaned)
- `.join()` with factory dimensions for context
- `.groupBy().agg()` calculates KPIs: device counts, avg/max temperature, etc.
- These metrics automatically update as new sensor data arrives

✅ **Result:** Always-current factory KPIs ready for your dashboards, Genie spaces, and reports!

## Step 4: Explore the Datasets Created by Your Pipeline

In this step, you perform ad-hoc queries on the data processed in the ETL pipeline to analyze the sensor data in an exploration notebook.

### What is an Exploration Notebook?

Exploration notebooks let you perform ad-hoc queries and analysis on your pipeline data without affecting the pipeline itself. These are SQL notebooks stored in an `explorations` folder. Files in the `explorations` folder are **not** run as part of a pipeline update by default.

### Create an Exploration Notebook

1. From the **pipeline assets browser sidebar**, click Plus icon **Add** then **Exploration**.
2. Enter a **Name**: `Sensor Analysis`
3. Select **SQL** for the exploration file.
4. A SQL notebook is created in a new `explorations` folder.

The SQL notebook has cells that you can run together or separately.

### Query 1: Find High-Temperature Factories

Let's find which factories have the highest average temperatures.

1. Copy the following code into the first cell of your exploration notebook:

```sql
-- Which factories have the highest average temperature?
SELECT 
  factory_name,
  region,
  city,
  avg_temperature,
  max_temperature,
  total_devices
FROM factory_kpis_gold
WHERE avg_temperature > 75
ORDER BY avg_temperature DESC;
```

2. Click Play icon or press `Shift + Enter` to run this query.

### Query 2: Analyze Recent Critical Readings

Let's find recent sensor readings that exceeded safe temperature thresholds.

1. Add the following code to the next cell:

```sql
-- Find recent critical temperature readings
SELECT 
  device_id,
  factory_id,
  timestamp,
  temperature,
  air_pressure,
  rotation_speed
FROM sensor_silver
WHERE temperature > 85
  AND timestamp >= current_date() - INTERVAL 7 DAYS
ORDER BY temperature DESC
LIMIT 100;
```

2. Click Play icon or press `Shift + Enter` to run this query.

### What You'll See

The pipeline assets browser shows:
- **Pipeline graph (DAG):** Your streaming table and materialized view with data flow
- **Exploration notebooks:** Separate from pipeline logic for ad-hoc analysis
- **Table preview:** Sample data from your tables
- **Query results:** Results from your exploration queries

---

**Tip:** You can create multiple exploration notebooks to organize different types of analysis (e.g., data quality checks, performance analysis, anomaly detection).

## Step 5: Create a Job to Run the Pipeline

Next, create a workflow to automate data ingestion, processing, and analysis steps using a Databricks job that runs on a schedule.

### Schedule Your Pipeline

1. At the top of the editor, choose the **Schedule** button.
2. If the **Schedules** dialog appears, choose **Add schedule**.
3. This opens the **New schedule** dialog, where you can create a job to run your pipeline on a schedule.
4. Optionally, give the job a name: `IoT Sensor Pipeline - Daily Update`
5. By default, the schedule is set to run once per day. You can accept this default, or set your own schedule:
   - Choosing **Advanced** gives you the option to set a specific time that the job will run
   - Selecting **More options** allows you to create notifications when the job runs
6. Select **Create** to apply the changes and create the job.

Now the job will run daily to keep your pipeline up to date. You can choose **Schedule** again to view the list of schedules. You can manage schedules for your pipeline from that dialog, including adding, editing, or removing schedules.

Clicking the name of the schedule (or job) takes you to the job's page in the **Jobs & pipelines** list. From there you can view details about job runs, including the history of runs, or run the job immediately with the **Run now** button.

### Monitoring Your Pipeline

After deployment, you get automatic observability:
- ✅ **Pipeline graph** - Visual data flow showing nodes and edges
- ✅ **Data quality dashboard** - Expectation pass/fail rates
- ✅ **Event log** - Detailed execution history
- ✅ **Performance metrics** - Processing speed, row counts, cluster utilization
- ✅ **Lineage tracking** - Understand data dependencies

---

## Complete Pipeline Code

Here's everything in one file you can copy/paste into the Lakeflow Pipeline Editor. This code includes both the streaming table and materialized view in a single source file.

**When to use this:**
- If you want all pipeline logic in one place
- For quick prototyping and testing
- When your pipeline is relatively simple

**Alternative approach:**
- You can split this into multiple files (e.g., `silver_layer.py` and `gold_layer.py`)
- Use the Plus icon **Add** then **Source** to create additional files
- All files in the `transformations` folder are processed together

---

### Python Complete Pipeline

**Instructions:**
1. Copy the entire code block below
2. Paste into your pipeline source file in the Lakeflow Pipelines Editor
3. Click Play icon **Run file** or **Run pipeline** to execute
4. View the pipeline graph on the right to see `sensor_silver` and `factory_kpis_gold` nodes

```python
"""
IoT Aircraft Sensor Pipeline
Complete declarative pipeline for production-ready sensor data processing

Configuration:
- READ_CATALOG: dwx_airops_insights_platform_dev_working
- READ_SCHEMA: db_crash_course (shared read schema)
- WRITE_SCHEMA: <your_username> (personal write schema, special chars replaced with _)
"""

from pyspark import pipelines as dp
from pyspark.sql.functions import (
    col, when, abs as abs_func, current_timestamp,
    avg, max, count, countDistinct, round as spark_round
)
import re

# Configuration
READ_CATALOG = "dwx_airops_insights_platform_dev_working"
READ_SCHEMA = "db_crash_course"
username = spark.sql("SELECT current_user()").collect()[0][0]
WRITE_SCHEMA = re.sub(r'[@\.]', '_', username)

# ========================================
# SILVER LAYER - Cleaned Sensor Data
# ========================================

@dp.table(
    name="sensor_silver",
    comment="Cleaned and validated aircraft sensor readings",
    expect={
        "valid_device_id": "device_id IS NOT NULL",
        "valid_timestamp": "timestamp IS NOT NULL",
        "valid_temperature_range": "temperature BETWEEN -50 AND 150"
    },
    expect_or_drop={
        "positive_air_pressure": "air_pressure > 0"
    }
)
def sensor_silver():
    """Ingest and clean sensor data with Auto Loader."""
    return (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("cloudFiles.inferColumnTypes", "true")
        .option("header", "true")
        .load(f"/Volumes/{READ_CATALOG}/{READ_SCHEMA}/sensor_data/")
        .withColumn("air_pressure",
                   when(col("air_pressure") < 0, abs_func(col("air_pressure")))
                   .otherwise(col("air_pressure")))
        .withColumn("processed_at", current_timestamp())
    )

# ========================================
# GOLD LAYER - Factory KPIs
# ========================================

@dp.materialized_view(
    name="factory_kpis_gold",
    comment="Factory-level KPIs for aircraft maintenance dashboards"
)
def factory_kpis_gold():
    """Aggregate factory-level metrics."""
    sensors = spark.read.table("sensor_silver")
    factories = spark.read.table(f"{READ_CATALOG}.{READ_SCHEMA}.dim_factories")
    
    enriched = sensors.join(factories, "factory_id", "left")
    
    return (
        enriched
        .groupBy("factory_id", "factory_name", "region", "city")
        .agg(
            countDistinct("device_id").alias("total_devices"),
            count("*").alias("total_readings"),
            spark_round(avg("temperature"), 2).alias("avg_temperature"),
            spark_round(max("temperature"), 2).alias("max_temperature"),
            spark_round(avg("air_pressure"), 2).alias("avg_air_pressure"),
            spark_round(avg("rotation_speed"), 2).alias("avg_rotation_speed")
        )
    )

print("✅ Pipeline defined! Deploy in the Lakeflow Pipelines Editor.")
print(f"📖 Reading from: {READ_CATALOG}.{READ_SCHEMA}")
print(f"✍️  Writing to: {READ_CATALOG}.{WRITE_SCHEMA}")
```

### SQL Complete Pipeline

**Instructions:**
1. Copy the entire code block below
2. Paste into your pipeline source file in the Lakeflow Pipelines Editor
3. Click Play icon **Run file** or **Run pipeline** to execute
4. View the pipeline graph on the right to see `sensor_silver` and `factory_kpis_gold` nodes

```sql
-- IoT Aircraft Sensor Pipeline (SQL Version)
-- 
-- Configuration:
-- READ_CATALOG: dwx_airops_insights_platform_dev_working
-- READ_SCHEMA: db_crash_course (shared)
-- WRITE_SCHEMA: <your_username> (personal, special chars replaced with _)
--
-- Note: Update the target catalog and schema in the pipeline settings to:
-- Target: dwx_airops_insights_platform_dev_working.<your_username>

-- SILVER LAYER: Cleaned sensor data
CREATE OR REFRESH STREAMING TABLE sensor_silver (
  CONSTRAINT valid_device_id EXPECT (device_id IS NOT NULL),
  CONSTRAINT valid_timestamp EXPECT (timestamp IS NOT NULL),
  CONSTRAINT valid_temperature_range EXPECT (temperature BETWEEN -50 AND 150),
  CONSTRAINT positive_air_pressure EXPECT OR DROP (air_pressure > 0)
)
COMMENT 'Cleaned and validated aircraft sensor readings'
AS SELECT
  device_id,
  trip_id,
  factory_id,
  model_id,
  timestamp,
  airflow_rate,
  rotation_speed,
  CASE 
    WHEN air_pressure < 0 THEN ABS(air_pressure)
    ELSE air_pressure
  END as air_pressure,
  temperature,
  delay,
  density,
  current_timestamp() as processed_at
FROM STREAM read_files(
  '/Volumes/dwx_airops_insights_platform_dev_working/db_crash_course/sensor_data/',
  format => 'csv',
  header => 'true'
);

-- GOLD LAYER: Factory KPIs
CREATE OR REFRESH MATERIALIZED VIEW factory_kpis_gold
COMMENT 'Factory-level KPIs for aircraft maintenance dashboards'
AS SELECT
  s.factory_id,
  f.factory_name,
  f.region,
  f.city,
  COUNT(DISTINCT s.device_id) as total_devices,
  COUNT(*) as total_readings,
  ROUND(AVG(s.temperature), 2) as avg_temperature,
  ROUND(MAX(s.temperature), 2) as max_temperature,
  ROUND(AVG(s.air_pressure), 2) as avg_air_pressure,
  ROUND(AVG(s.rotation_speed), 2) as avg_rotation_speed
FROM sensor_silver s
LEFT JOIN dwx_airops_insights_platform_dev_working.db_crash_course.dim_factories f 
  ON s.factory_id = f.factory_id
GROUP BY s.factory_id, f.factory_name, f.region, f.city;
```

## Summary

🎉 **Congratulations!** You've built a production-ready data pipeline with Lakeflow Spark Declarative Pipelines!

### What You Accomplished

✅ **Created a pipeline** using the new Lakeflow Pipelines Editor  
✅ **Built a streaming table** - Automatically ingests and cleans sensor data with Auto Loader  
✅ **Added data quality** - Tracks and enforces expectations  
✅ **Created a materialized view** - Always-current factory KPIs  
✅ **Performed ad-hoc analysis** - Created exploration notebooks for queries  
✅ **Scheduled automation** - Set up a job to run your pipeline daily  

### What You Get For Free

The Lakeflow framework automatically provides:
- ✅ **Incremental processing** - Only processes new data
- ✅ **Checkpoint management** - Exactly-once guarantees
- ✅ **Schema evolution** - Handles new columns gracefully
- ✅ **Data quality tracking** - Expectations dashboard
- ✅ **Lineage visualization** - See data flow between tables
- ✅ **Error recovery** - Automatic retries
- ✅ **Monitoring** - Performance and health metrics

### Key Takeaways

1. **Declarative approach** - You define WHAT you want, not HOW to get it
2. **Auto Loader** - Automatically discovers and processes new files
3. **Streaming tables** - For append-only incremental processing
4. **Materialized views** - For aggregations that update
5. **Exploration notebooks** - For ad-hoc queries separate from pipeline logic
6. **Built-in observability** - Automatic monitoring and data quality tracking

### Next Steps for Your Week

Now that you have clean, aggregated data:

1. **Dashboards** (Day 1) - Use `factory_kpis_gold` for visualizations
2. **Genie** (Day 1) - Connect to `sensor_silver` for natural language queries
3. **AutoML** (Day 2) - Use `sensor_silver` for predictive maintenance models
4. **AI Agents** (Day 2) - Build on clean data for AI systems

You're well on your way to meeting leadership's deadline! 🚀

---

**Additional Resources:**
- [Lakeflow Spark Declarative Pipelines](https://learn.microsoft.com/en-us/azure/databricks/ldp/)
- [What is Auto Loader?](https://learn.microsoft.com/en-us/azure/databricks/ingestion/auto-loader/)
- [Pipeline Development Guide](https://learn.microsoft.com/en-us/azure/databricks/ldp/develop)
- [Multi-File Editor](https://learn.microsoft.com/en-us/azure/databricks/ldp/multi-file-editor)

## Try This Out: Extend Your Pipeline

Want to learn more? Here are some ideas to explore:

### 1. Add Device-Level Metrics

Create another materialized view for per-device KPIs to track individual device health.

**Steps:**
1. In your pipeline editor, add the following code to your source file (or create a new source file)
2. Click Play icon **Run file** to update the pipeline

**Python:**
```python
@dp.materialized_view(name="device_health_gold")
def device_health_gold():
    return (
        spark.read.table("sensor_silver")
        .groupBy("device_id", "factory_id")
        .agg(
            avg("temperature").alias("avg_temp"),
            count("*").alias("reading_count")
        )
    )
```

**SQL:**
```sql
CREATE OR REFRESH MATERIALIZED VIEW device_health_gold
AS SELECT 
  device_id,
  factory_id,
  AVG(temperature) as avg_temp,
  COUNT(*) as reading_count
FROM sensor_silver
GROUP BY device_id, factory_id;
```

The pipeline graph will update to show three nodes: `sensor_silver` → `factory_kpis_gold` and `device_health_gold`.

### 2. Add Inspection Data Pipeline

Create a parallel streaming table for inspection records.

**Steps:**
1. From the pipeline assets browser, click Plus icon **Add** then **Source**
2. Choose your language (Python or SQL)
3. Paste the code below
4. Click Play icon **Run file**

**Python:**
```python
@dp.table(name="inspection_silver")
def inspection_silver():
    return (
        spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "csv")
        .option("header", "true")
        .load("/Volumes/dwx_airops_insights_platform_dev_working/db_crash_course/inspection_data/")
    )
```

**SQL:**
```sql
CREATE OR REFRESH STREAMING TABLE inspection_silver
AS SELECT *
FROM STREAM read_files(
  '/Volumes/dwx_airops_insights_platform_dev_working/db_crash_course/inspection_data/',
  format => 'csv',
  header => 'true'
);
```

### 3. Organize with Multiple Files

Use the multi-file editor to organize your pipeline by layer:

**Steps:**
1. Click Plus icon **Add** then **Source** to create new files
2. Name them: `bronze_layer.py`, `silver_layer.py`, `gold_layer.py`
3. Move related code to each file
4. Run Play icon **Run pipeline** to process all files together

Benefits:
- Better organization
- Easier collaboration
- Clear separation of concerns
- All files in the `transformations` folder are processed together

### 4. Create More Exploration Notebooks

Practice ad-hoc queries with additional exploration notebooks:

**Ideas:**
- **Data Quality Report**: Check for data issues and anomalies
- **Performance Analysis**: Analyze sensor performance trends
- **Factory Comparison**: Compare metrics across regions

**Steps:**
1. Click Plus icon **Add** then **Exploration**
2. Name your notebook
3. Write SQL queries to analyze your data
4. These don't affect the pipeline, only for analysis

### 5. Explore Advanced Features

- **Change Data Capture (CDC)** - Handle updates and deletes
- **Watermarks** - Handle late-arriving data
- **Flows** - Multiple sources writing to one table
- **Expectations with actions** - Drop or quarantine bad data

**Documentation:**
- [Streaming Tables](https://learn.microsoft.com/en-us/azure/databricks/ldp/streaming-tables)
- [Materialized Views](https://learn.microsoft.com/en-us/azure/databricks/ldp/materialized-views)
- [Multi-File Editor](https://learn.microsoft.com/en-us/azure/databricks/ldp/multi-file-editor)
- [Data Quality Expectations](https://learn.microsoft.com/en-us/azure/databricks/ldp/expectations)